<a href="https://colab.research.google.com/github/jrhumberto/cd/blob/main/003_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fontes:
- https://github.com/aasouzaconsult/Cientista-de-Dados/tree/master/NLP%20-%20Classifica%C3%A7%C3%A3o%20de%20Not%C3%ADcias%20Curtas%20PTB
- https://medium.com/blog-do-zouza/classifica%C3%A7%C3%A3o-de-not%C3%ADcias-utilizando-machine-learning-b25ff63ea51f

In [ ]:
from time import time
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import gensim
import pickle
from gensim.models.word2vec import Word2Vec
from gensim.models import FastText
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import average_precision_score
from sklearn import metrics
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import RandomForestClassifier

In [ ]:
class E2V_AVG(object):
    def __init__(self, word2vec):
        self.w2v = word2vec
        self.dimensao = 300
    
    def fit(self, X, y):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.w2v[word] for word in words if word in self.w2v] or [np.zeros(self.dimensao)], axis=0)
            for words in X
        ])

In [ ]:
# ReferÊncia (SOUZA, 2019)
class E2V_IDF(object):
    def __init__(self, word2vec):
        self.w2v = word2vec
        self.wIDF = None # IDF da palavra na colecao
        self.dimensao = 300
        
    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        maximo_idf = max(tfidf.idf_) # Uma palavra que nunca foi vista (rara) então o IDF padrão é o máximo de idfs conhecidos (exemplo: 9.2525763918954524)
        self.wIDF = defaultdict(
            lambda: maximo_idf, 
            [(word, tfidf.idf_[i]) for word, i in tfidf.vocabulary_.items()])
        return self
    
    # Gera um vetor de 300 dimensões, para cada documento, com a média dos vetores (embeddings) dos termos * IDF, contidos no documento.
    def transform(self, X):
        return np.array([
                np.mean([self.w2v[word] * self.wIDF[word] for word in words if word in self.w2v] or [np.zeros(self.dimensao)], axis=0)
                for words in X
            ])


In [ ]:
# Arquivo com nóticias curtas em Português do site G1
X = pickle.load(open('/data/z6News_X.ipy', 'rb'))
# Arquivo com o rótulos das notícias
y = pickle.load(open('/data/z6News_y.ipy', 'rb'))

# Essa fonte de dados é própria e esta disponível aqui no GitHub na Pasta: data
# - Podem utilizar, bastando referenciar o autor: SOUZA, 2019 (descrito na seção Referências)


In [ ]:


# Tranformando em Array
X, y = np.array(X), np.array(y)



In [ ]:


print ("Total de Notícias - G1: %s" % len(y))



In [ ]:


model = Word2Vec(X, size=300, window=5, sg=1, workers=4)
w2v = {w: vec for w, vec in zip(model.wv.index2word, model.wv.vectors)}

